In [1]:
import csv
import math
import string
import itertools
from io import open
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Iterable, defaultdict
import random

In [2]:
# set determinstic results
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()

from encoder import *
from decoder import *
from emb2seq_model import *

# get the decoder vocab
with open('./data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
    print("Size of vocab: {}".format(vocab.idx))

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Size of vocab: 49036


In [4]:
decoder = Decoder(vocab_size = vocab.idx)
encoder = Encoder(elmo_class = elmo)
emb2seq_model = Emb2Seq_Model(encoder, decoder, vocab = vocab)

# randomly initialize the weights
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)       
emb2seq_model.apply(init_weights)

Emb2Seq_Model(
  (encoder): Encoder(
    (dimension_reduction): Linear(in_features=3072, out_features=512, bias=True)
    (lstm): LSTM(512, 256, num_layers=2, bidirectional=True)
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=300, bias=True)
      (1): ReLU()
      (2): Dropout(p=0)
      (3): Linear(in_features=300, out_features=256, bias=True)
      (4): Dropout(p=0)
    )
  )
  (decoder): Decoder(
    (lstm_cell): LSTMCell(512, 256)
    (linear): Linear(in_features=256, out_features=49036, bias=True)
  )
  (embed): Embedding(49036, 256, padding_idx=0)
  (dropout): Dropout(p=0)
)

In [5]:
# training hyperparameters
optimizer = optim.Adam(emb2seq_model.parameters(), lr = 0.1)
PAD_IDX = vocab('<pad>')
print('PAD_IDX: {}'.format(PAD_IDX))
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {}'.format(device))
print(torch.cuda.device_count())
emb2seq_model.to(device)


PAD_IDX: 0
Device: cpu
0


Emb2Seq_Model(
  (encoder): Encoder(
    (dimension_reduction): Linear(in_features=3072, out_features=512, bias=True)
    (lstm): LSTM(512, 256, num_layers=2, bidirectional=True)
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=300, bias=True)
      (1): ReLU()
      (2): Dropout(p=0)
      (3): Linear(in_features=300, out_features=256, bias=True)
      (4): Dropout(p=0)
    )
  )
  (decoder): Decoder(
    (lstm_cell): LSTMCell(512, 256)
    (linear): Linear(in_features=256, out_features=49036, bias=True)
  )
  (embed): Embedding(49036, 256, padding_idx=0)
  (dropout): Dropout(p=0)
)

In [6]:
# utility function
# turn the given definition into its index list form
def def2idx(definition, max_length, vocab):
    
    # definition is given by the WN NLTK API in a string
    def_tokens = nltk.tokenize.word_tokenize(definition.lower())
    
    # limit the length if too long, trim
    if len(def_tokens) > (max_length - 2):
        def_tokens = def_tokens[0:(max_length - 2)]
        
        # add the start and end symbol
        def_tokens = ['<start>'] + def_tokens + ['<end>']
    
    # if the length is too short, pad
    elif len(def_tokens) < (max_length - 2):
        
        # add the start and end symbol
        def_tokens = ['<start>'] + def_tokens + ['<end>']
        
        pad = ['<pad>'] * (max_length - len(def_tokens))
        def_tokens = def_tokens + pad
        
    else:
        def_tokens = ['<start>'] + def_tokens + ['<end>']
            
    # get the index for each element in the token list
    def_idx_list = [vocab(token) for token in def_tokens]
    
    return def_idx_list
  

In [7]:
# parse the SemCor training data
import xml.etree.ElementTree as ET
tree = ET.parse('../WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml')
corpus = tree.getroot()

# parse the target sense tag 
target_file = open("../WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt", "r")

# small sets of SemCor
small_train_size = 1
small_dev_size = 2

In [8]:
# the training function
def train(model, optimizer, corpus, criterion, clip):
    
    model.train()
    epoch_loss = 0
    sentence_num = 0
    
    for sub_corpus in corpus[0:small_train_size]:
    
        for sent in sub_corpus[0:20]:

            optimizer.zero_grad()
            
            # get the plain text sentence
            sentence = [word.text for word in sent]
            
            # get the tagged ambiguous words
            tagged_sent = [instance for instance in sent if instance.tag == 'instance']
            # print(sentence)
            # print(tagged_sent)
            
            # only use sentence with at least one tagged word
            if len(tagged_sent) > 0:
                
                sentence_num += 1
                
                # get all-word definitions, batch_size is the sentence length
                # [batch_size, self.max_length]
                definitions = []
                for instance in tagged_sent:
                    
                    # get the sense from the WN
                    # senses are in-order already
                    key = target_file.readline().replace('\n', '').split(' ')[-1]
                    definition = wn.lemma_from_key(key).synset().definition()                 
                    def_idx_list = def2idx(definition, model.max_length, vocab)
                    definitions.append(def_idx_list)

                # get the encoder-decoder result
                # (self.max_length, batch_size, vocab_size)
                output, result = model(sentence, tagged_sent, definitions, teacher_forcing_ratio = 0.4)

                # adjust dimension for loss calculation
                
                output = output.permute(1, 2, 0)
                # print(output.shape)
                target = torch.tensor(definitions, dtype = torch.long).to(device)

                loss = criterion(output, target).to(device)
                loss.backward()

                # add clip for gradient boost
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

                optimizer.step()
                epoch_loss += loss.item()
        
    return epoch_loss / sentence_num, result

In [9]:
# evaluate the model
def evaluate(model, corpus, criterion):
    
    model.eval()
    epoch_loss = 0
    sentence_num = 0
        
    with torch.no_grad():
    
        for sub_corpus in corpus[small_train_size:small_dev_size]:
    
            for sent in sub_corpus[0:5]:
            
                sentence = [word.text for word in sent]
                
                # get the tagged ambiguous words
                tagged_sent = [instance for instance in sent if instance.tag == 'instance']
                # print(sentence)
                # print(tagged_sent)

                # only use sentence with at least one tagged word
                if len(tagged_sent) > 0:
                    sentence_num += 1

                    # get all-word definitions, batch_size is the sentence length
                    # [batch_size, self.max_length]
                    definitions = []
                    for instance in tagged_sent:

                        # get the sense from the WN
                        # senses are in-order already
                        key = target_file.readline().replace('\n', '').split(' ')[-1]
                        definition = wn.lemma_from_key(key).synset().definition()                 
                        def_idx_list = def2idx(definition, model.max_length, vocab)
                        definitions.append(def_idx_list)

                    # get the encoder-decoder result
                    # (self.max_length, batch_size, vocab_size)
                    output, result = model(sentence, tagged_sent, definitions, teacher_forcing_ratio = 0.4)

                    # adjust dimension for loss calculation
                    output = output.permute(1, 2, 0)
                    # print(output.shape)
                    target = torch.tensor(definitions, dtype = torch.long).to(device)

                    loss = criterion(output, target).to(device)         
                    epoch_loss += loss.item()
                            
    return epoch_loss / sentence_num , result

In [10]:
# time used by each epoch
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
# train and evaluate
import time

N_EPOCHS = 50
CLIP = 1
best_valid_loss = float('inf')
train_losses = []
dev_losses = []

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, _ = train(emb2seq_model, optimizer, corpus, criterion, CLIP)
    train_losses.append(train_loss)
    
    valid_loss, result = evaluate(emb2seq_model, corpus, criterion)
    dev_losses.append(valid_loss)
        
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # visualize the results
    all_results = []
    for n in range(len(result[0])):
        sense = ''
        for m in range(len(result)):
            w = ' '+ vocab.idx2word.get(int(result[m][n]))
            sense += w
        all_results.append(sense)
    with open('result.txt', 'w') as f:
        for item in all_results:
            f.write("%s\n" % item)
            
    # save the best model based on the dev set
    '''
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(seq2seq_model.state_dict(), 'best_model.pth')
    '''
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 54s
	Train Loss: 22.760 | Train PPL: 7664783805.655
	 Val. Loss: 31.325 |  Val. PPL: 40204421483268.617
Epoch: 02 | Time: 1m 1s
	Train Loss: 29.199 | Train PPL: 4796943101505.502
	 Val. Loss: 29.015 |  Val. PPL: 3992498978990.643
Epoch: 03 | Time: 1m 38s
	Train Loss: 32.386 | Train PPL: 116129395729019.844
	 Val. Loss: 32.071 |  Val. PPL: 84790964154449.344
Epoch: 04 | Time: 0m 59s
	Train Loss: 37.676 | Train PPL: 23044581610280236.000
	 Val. Loss: 41.080 |  Val. PPL: 693071466732036864.000
Epoch: 05 | Time: 0m 46s
	Train Loss: 43.860 | Train PPL: 11171624115129856000.000
	 Val. Loss: 51.323 |  Val. PPL: 19458912867365127454720.000
Epoch: 06 | Time: 0m 46s
	Train Loss: 53.353 | Train PPL: 148187069712185117638656.000
	 Val. Loss: 57.871 |  Val. PPL: 13580603786543451850407936.000
Epoch: 07 | Time: 0m 46s
	Train Loss: 58.382 | Train PPL: 22637118137429217012350976.000
	 Val. Loss: 57.594 |  Val. PPL: 10295689533501119976701952.000
Epoch: 08 | Time: 0m 46s
	Train Los

Epoch: 44 | Time: 1m 2s
	Train Loss: 139.264 | Train PPL: 3029880528718785908419237965466249108688502713857956007378944.000
	 Val. Loss: 163.622 |  Val. PPL: 114808152313008490313126386611800604222057753992453587793571837477126144.000
Epoch: 45 | Time: 1m 2s
	Train Loss: 167.973 | Train PPL: 8907491939700921254611915715450334506620681994005771984718372973920124928.000
	 Val. Loss: 140.122 |  Val. PPL: 7147875621788137475046604243673571531730158375837346056634368.000
Epoch: 46 | Time: 1m 2s
	Train Loss: 148.815 | Train PPL: 42599678516369389185329446477521660706731361900391376416816496640.000
	 Val. Loss: 141.605 |  Val. PPL: 31494048616182994243945664398812633675281598752193966841528320.000
Epoch: 47 | Time: 1m 2s
	Train Loss: 145.160 | Train PPL: 1102135448836937527498478672179370022843574442670925140117684224.000
	 Val. Loss: 111.773 |  Val. PPL: 3485337627883243794914761175262401131868319645696.000
Epoch: 48 | Time: 1m 6s
	Train Loss: 139.910 | Train PPL: 578078067234928886238684684

In [12]:
# plot the learning curve
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc

with open('train_loss.tsv', mode = 'w') as loss_file:
    csv_writer = csv.writer(loss_file)
    csv_writer.writerow(train_losses)

with open('dev_loss.tsv', mode = 'w') as loss_file: 
    csv_writer = csv.writer(loss_file)
    csv_writer.writerow(dev_losses)

In [13]:
plt.figure(1)
# rc('text', usetex = True)
rc('font', family='serif')
plt.grid(True, ls = '-.',alpha = 0.4)
plt.plot(train_losses, ms = 4, marker = 's', label = "Train Loss")
plt.legend(loc = "best")
title = "CrossEntropy Loss"
plt.title(title)
plt.ylabel('Loss')
plt.xlabel('Number of Iteration')
plt.tight_layout()
plt.savefig('train_loss.png')

In [14]:
plt.figure(2)
# rc('text', usetex = True)
rc('font', family='serif')
plt.grid(True, ls = '-.',alpha = 0.4)
plt.plot(dev_losses, ms = 4, marker = 'o', label = "Dev Loss")
plt.legend(loc = "best")
title = "CrossEntropy Loss"
plt.title(title)
plt.ylabel('Loss')
plt.xlabel('Number of Iteration')
plt.tight_layout()
plt.savefig('dev_loss.png')